In [1]:
# The basics
import numpy as np
import pandas as pd

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Softmax, Conv1D, MaxPooling1D, GlobalMaxPooling1D

# utils
import os

In [2]:
isear = pd.read_csv('../data/raw/isear.csv', sep='|', error_bad_lines=False, usecols=['Field1', 'SIT', 'EMOT'])

In [3]:
number_of_classes = len(isear.EMOT.unique())

In [4]:
maxlen = 1000
max_words = 10000

In [5]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(isear['SIT'])
sequences = tokenizer.texts_to_sequences(isear['SIT'])

In [6]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9063 unique tokens.


In [7]:
data = pad_sequences(sequences, maxlen=maxlen, padding='post')

In [8]:
x_train, x_test, y_train, y_test = train_test_split(data, isear['EMOT'])

In [9]:
glove_dir = '../data/external'
embeddings_index = {}

f = open(os.path.join(glove_dir, 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [10]:
embeddings_index['king'] - embeddings_index['man'] + embeddings_index['woman'] - embeddings_index['queen']

array([ 0.03882596, -0.91902995,  0.25977004,  0.04227898,  0.13896999,
        0.20638007,  0.02683   , -0.03402001, -0.09974198,  0.08821005,
        0.007544  , -0.36111003,  0.320001  , -0.6568099 ,  0.63689005,
        0.33902   , -0.7714    , -0.59767103,  0.711054  ,  0.44857004,
       -0.08957994, -0.01644999, -0.08847399, -0.17911002,  0.43223003,
       -0.28219986, -0.09410012, -0.9570599 ,  0.01775998,  0.32842597,
        0.11489999, -0.36365002, -0.52906007,  0.11805284, -0.10499001,
       -0.219787  , -0.33020002, -0.137476  ,  0.10705006,  0.92508006,
       -0.223916  ,  0.71121997, -0.29156998,  0.24670503, -0.05374995,
        0.2274299 , -0.09986001,  0.33875012, -0.69724536,  0.10499996],
      dtype=float32)

In [11]:
embedding_dim = 50 # if chaning this, update the file name above 

embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## Model creation time

In [12]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(number_of_classes + 1,  activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 50)          500000    
_________________________________________________________________
conv1d (Conv1D)              (None, 994, 32)           11232     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 198, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 192, 32)           7200      
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 8)                 264       
Total params: 518,696
Trainable params: 518,696
Non-trainable params: 0
__________________________________________________

In [13]:
model.fit(x_train, to_categorical(y_train),
          epochs=10,
          batch_size=32,
          validation_data=(x_test, to_categorical(y_test)))

Train on 5749 samples, validate on 1917 samples
Epoch 1/10
5749/5749 [==============================] - 11s 2ms/sample - loss: 1.9708 - accuracy: 0.1465 - val_loss: 1.9396 - val_accuracy: 0.1748
Epoch 2/10
5749/5749 [==============================] - 11s 2ms/sample - loss: 1.8648 - accuracy: 0.2181 - val_loss: 1.7654 - val_accuracy: 0.2728
Epoch 3/10
5749/5749 [==============================] - 11s 2ms/sample - loss: 1.4775 - accuracy: 0.4343 - val_loss: 1.4799 - val_accuracy: 0.4413
Epoch 4/10
5749/5749 [==============================] - 11s 2ms/sample - loss: 0.9808 - accuracy: 0.6624 - val_loss: 1.4638 - val_accuracy: 0.4982
Epoch 5/10
5749/5749 [==============================] - 11s 2ms/sample - loss: 0.6602 - accuracy: 0.7963 - val_loss: 1.5872 - val_accuracy: 0.5013
Epoch 6/10
5749/5749 [==============================] - 11s 2ms/sample - loss: 0.4370 - accuracy: 0.8768 - val_loss: 1.7765 - val_accuracy: 0.5076
Epoch 7/10
5749/5749 [==============================] - 11s 2ms/sample

In [14]:
y_pred = model.predict_classes(x_test)
y_pred

array([4, 7, 1, ..., 7, 2, 5])

In [15]:
confusion_matrix(y_test, y_pred)

array([[156,  20,   7,  35,   8,  47,  13],
       [ 11, 147,  35,  22,  15,  24,   7],
       [  7,  22, 129,  22,  31,  41,  40],
       [ 18,  16,  23, 142,  15,  36,  12],
       [ 16,  19,  28,  15, 144,  29,  23],
       [ 19,  22,  30,  15,  31, 106,  39],
       [ 12,   6,  49,  15,  12,  49, 137]])